In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import roc_auc_score

Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [2]:
data = pd.read_csv('data-logistic.csv', header=None)
y = data[0]
X = data.loc[:, 1:]

Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [3]:
def funk_w1(w1, w2, y, X, k, C):
    lenght = len(y)
    S = 0
    for i in range(0, lenght):
        S += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))

    return w1 + (k * (1.0 / lenght) * S) - k * C * w1


def funk_w2(w1, w2, y, X, k, C):
    lenght = len(y)
    S = 0
    for i in range(0, lenght):
        S += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))

    return w2 + (k * (1.0 / lenght) * S) - k * C * w2

Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [4]:
def grad(y, X, C=0.0, w1=0.0, w2=0.0, k=0.1, err=1e-5):
    i = 0
    i_max = 10000
    w1_new, w2_new = w1, w2

    while True:
        i += 1
        w1_new = funk_w1(w1, w2, y, X, k, C)
        w2_new = funk_w2(w1, w2, y, X, k, C)
        e = math.sqrt((w1_new - w1) ** 2 + (w2_new - w2) ** 2)

        if i >= i_max or e <= err:
            break
        else:
            w1, w2 = w1_new, w2_new

    return [w1_new, w2_new]

Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [5]:
w1, w2 = grad(y, X)
l2_w1, l2_w2 = grad(y, X, 10.0)

Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 



In [6]:
def a(X, w1, w2):
    return 1.0 / (1.0 + math.exp(-w1 * X[1] - w2 * X[2]))

In [7]:
y_score = data.apply(lambda x: a(x, w1, w2), axis=1)
y_rscore = data.apply(lambda x: a(x, l2_w1, l2_w2), axis=1)

auc = roc_auc_score(y, y_score)
rauc = roc_auc_score(y, y_rscore)

print("{:0.3f} {:0.3f}".format(auc, rauc))

0.927 0.936
